# Zonal stats along line segments

This quick example shows off using {py:func}`~rasterix.rasterize.rasterize`, and Xarray's GroupBy to do zonal stats along a line.

## Example dataset

Here is an example dataset in the EPSG:3035 coordinate system with a coordinate system specified using a `"GeoTransform'` attribute.

In [ ]:
import numpy as np
import pyproj
import shapely
import xarray as xr

ds = xr.Dataset(
    coords={
        "spatial_ref": (
            (),
            0,
            pyproj.CRS.from_epsg(3035).to_cf() | {"GeoTransform": "100000 12000 0 60000 0 -12000"},
        )
    },
)
ds["foo"] = xr.DataArray(
    np.random.default_rng(0).integers(0, 256, size=(100, 200), dtype=np.uint8),
    dims=("y", "x"),
)
ds

## Generate coordinates with `RasterIndex`

We'll use {py:func}`~rasterix.assign_index` to assign lazy coordinate locations in `x` and `y`.

In [ ]:
import rasterix

ds = rasterix.assign_index(ds)
ds

## Make a geometry

We will take a LineString and split it in to 80km segments.

For simplicity the line will extend from the top-left corner to the bottom-right corner.

In [ ]:
import geopandas as gpd

from rasterix.rasterize import rasterize

# make a diagonal line across the raster
line = gpd.GeoDataFrame(
    {
        "geometry": [
            shapely.LineString(
                [
                    [ds.x[0].item(), ds.y[0].item()],
                    [ds.x[-1].item(), ds.y[-1].item()],
                ]
            )
        ]
    }
)
# 80km segments
line = line.segmentize(80_000)
coords = shapely.get_coordinates(line)
segment_coords = np.stack([coords[:-1], coords[1:]], axis=1)  # shape (n-1, 2, 2)
segments = shapely.linestrings(segment_coords)
geoms = gpd.GeoDataFrame({"geometry": segments})
geoms

## Rasterize the geometries

In [ ]:
res = (
    rasterize(
        ds,
        geoms,
        all_touched=True,
        engine="rasterio",
    )
    .compute()
    .rename("segment")
)
res.plot()

## Zonal stats with Xarray's GroupBy

(faster when powered by [flox](https://flox.readthedocs.io/en/latest/)!)

In [ ]:
zonal = ds["foo"].groupby(res).max()
# drop the last segment (background/NaN pixels)
zonal = zonal.isel(segment=slice(-1))
zonal

## Make a vector data cube with Xvec

In [ ]:
import xvec  # noqa: F401

zonal_ds = geoms.assign(mean_foo=zonal.values).set_geometry("geometry").set_index("geometry").to_xarray()
zonal_ds = zonal_ds.xvec.set_geom_indexes("geometry", crs=3035)
zonal_ds

## Plot

In [ ]:
f, ax = zonal_ds["mean_foo"].xvec.plot()
f.set_size_inches((8, 2))